# draphe dask a partier projet de clinic 


In [ ]:
from dash import Dash, html, callback, dcc
import pandas as pd
import plotly.graph_objects as go
from dash import dash_table
from dash import Output, Input
import plotly.express as px

# copie votre path 
df = pd.read_csv(r'C:\Users\dell\OneDrive\Desktop\cmc\2eme\visuilisation yassin\GraphRelationsAnalyzer\DATA\monthly_deaths.csv')
df_monthly_deaths = pd.read_csv(r'C:\Users\dell\OneDrive\Desktop\cmc\2eme\visuilisation yassin\GraphRelationsAnalyzer\DATA\monthly_deaths.csv')
df_annual_deaths_by_clinic = pd.read_csv(r'C:\Users\dell\OneDrive\Desktop\cmc\2eme\visuilisation yassin\GraphRelationsAnalyzer\DATA\annual_deaths_by_clinic.csv')


df_yearly = pd.DataFrame(df_annual_deaths_by_clinic)
df_monthly = pd.DataFrame(df_monthly_deaths)
df_monthly['date'] = pd.to_datetime(df_monthly['date'])
df_monthly['pct_deaths'] = df_monthly['deaths'] / df_monthly['births']


df_avant_la_date_1847 = df_monthly[df_monthly['date'] < '1847-06-01']
df_apres_la_date_1847 = df_monthly[df_monthly['date'] >= '1847-06-01']
df_avant_la_date_1847.loc[:, 'rolling_avg'] = df_avant_la_date_1847['pct_deaths'].rolling(window=6).mean()


app = Dash(__name__)


app.layout = html.Div(
    style={
        'font-family': 'Arial, sans-serif',
        'background-color': '#f9f9f9',
        'margin': 0,
        'padding': 0,
    },
    children=[
        html.H1(
            "HELLO WORLD",
            style={
                'color': 'green',
                'text-align': 'center',
                'background-color': 'white',
                'border': '2px solid black',
                'border-radius': '10px',
                'padding': '10px',
                'margin': '20px auto',
                'width': 'fit-content',
            },
        ),
        html.Div(
            style={'display': 'flex', 'justify-content': 'space-between'},
            children=[
                html.Div(
                    style={
                        'width': '45%',
                        'padding': '10px',
                        'border-radius': '10px',
                        'margin': '10px auto',
                        'box-shadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                        'border': '2px solid #ddd',
                    },
                    children=[
                        dash_table.DataTable(
                            data=df_monthly_deaths.to_dict('records'),
                            columns=[{'id': c, 'name': c} for c in df_monthly_deaths.columns],
                            page_size=10,
                            editable=True,
                            filter_action='native',
                            row_deletable=True,
                            style_table={'overflowX': 'scroll'},
                            style_cell={'textAlign': 'center'},
                        )
                    ],
                ),
                html.Div(
                    style={
                        'width': '45%',
                        'padding': '10px',
                        'border-radius': '10px',
                        'margin': '10px auto',
                        'box-shadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                        'border': '2px solid #ddd',
                    },
                    children=[
                        dcc.Dropdown(
                            id='dropdown-selection',
                            options=[
                                {'label': 'Before 1847', 'value': 'before_1847'},
                                {'label': 'After 1847', 'value': 'after_1847'},
                            ],
                            value='before_1847',
                            style={'width': '50%'}
                        ),
                        dcc.Graph(id='mortality-graph-before-after'),
                    ],
                ),
            ],
        ),
        html.Div(
            style={'display': 'flex', 'justify-content': 'space-between'},
            children=[
                html.Div(
                    style={
                        'width': '45%',
                        'padding': '10px',
                        'border-radius': '10px',
                        'margin': '10px auto',
                        'box-shadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                        'border': '2px solid #ddd',
                    },
                    children=[
                        dash_table.DataTable(
                            data=df_annual_deaths_by_clinic.to_dict('records'),
                            columns=[{'id': c, 'name': c} for c in df_annual_deaths_by_clinic.columns],
                            page_size=10,
                            editable=True,
                            filter_action='native',
                            row_deletable=True,
                            style_table={'overflowX': 'scroll'},
                            style_cell={'textAlign': 'center'},
                        )
                    ],
                ),
                html.Div(
                    style={
                        'width': '45%',
                        'padding': '10px',
                        'border-radius': '10px',
                        'margin': '10px auto',
                        'box-shadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                        'border': '2px solid #ddd',
                    },
                    children=[
                        dcc.Graph(id='graphiques-naissances-deces-cliniques'),
                    ],
                ),
            ],
        ),
        html.Div(
            style={'display': 'flex', 'justify-content': 'space-between'},
            children=[
                html.Div(
                    style={
                        'width': '45%',
                        'padding': '10px',
                        'border-radius': '10px',
                        'margin': '10px auto',
                        'box-shadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                        'border': '2px solid #ddd',
                    },
                    children=[
                        dcc.Graph(id='naissances-cliniques'),
                    ],
                ),
                html.Div(
                    style={
                        'width': '45%',
                        'padding': '10px',
                        'border-radius': '10px',
                        'margin': '10px auto',
                        'box-shadow': '0px 4px 6px rgba(0, 0, 0, 0.1)',
                        'border': '2px solid #ddd',
                    },
                    children=[
                        dcc.Graph(id='deces-cliniques'),
                    ],
                ),
            ],
        ),
    ],
)


@app.callback(
    Output('mortality-graph-before-after', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_mortality_graph(selection):
    if selection == 'before_1847':
        df_selected = df_avant_la_date_1847
    else:
        df_selected = df_apres_la_date_1847

    # Create a Plotly figure
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_selected['date'],
        y=df_selected['births'],
        mode='lines',
        name='Births',
        line=dict(color='skyblue', width=3)
    ))

    fig.add_trace(go.Scatter(
        x=df_selected['date'],
        y=df_selected['deaths'],
        mode='lines',
        name='Deaths',
        line=dict(color='crimson', width=2, dash='dash')
    ))

    fig.update_layout(
        title='Total Number of Monthly Births and Deaths',
        xaxis_title='Date',
        yaxis_title='Count',
        template='plotly_dark',
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True)
    )

    return fig


@app.callback(
    Output('graphiques-naissances-deces-cliniques', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_mortality_graph(selection):

    clinic_deaths = df_yearly.groupby('clinic')['deaths'].sum()
    clinic_births = df_yearly.groupby('clinic')['births'].sum()


    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=clinic_deaths.index,
        y=clinic_deaths.values,
        name='Deaths',
        marker=dict(color='crimson')
    ))

    fig.add_trace(go.Bar(
        x=clinic_births.index,
        y=clinic_births.values,
        name='Births',
        marker=dict(color='skyblue')
    ))

    fig.update_layout(
        title='Annual Births and Deaths by Clinic',
        barmode='stack',
        xaxis_title='Clinic',
        yaxis_title='Count',
        template='plotly_dark'
    )
    return fig


@app.callback(
    Output('naissances-cliniques', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_mortality_graph(selection):
    fig = px.line(df_yearly, x='year', y='births', color='clinic')

    fig.update_layout(
        template='plotly_dark',  # Set dark background
        title='Births by Clinic Over the Years',
        xaxis_title='Year',
        yaxis_title='Births',
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True)
    )
    return fig


@app.callback(
    Output('deces-cliniques', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_mortality_graph(selection):
    fig = px.line(df_yearly, x='year', y='deaths', color='clinic')


    fig.update_layout(
        template='plotly_dark',  # Set dark background
        title='Deaths by Clinic Over the Years',
        xaxis_title='Year',
        yaxis_title='Deaths',
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True)
    )

    return fig




if __name__ == '__main__':
 app.run(debug=True)


C:\Users\dell\AppData\Local\Temp\ipykernel_3376\104103952.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.